- Given a lat lon
- Assume maximum temperature
- convert lat lon to correct projection
- join all times together
- from megacube extract that geospatial point
    - investgate interopolation vs. find nearest
- collapse time by mean
- return float

In [1]:
lat = 52
lon = -2

In [2]:
import datetime
import iris
iris.FUTURE.netcdf_promote = True

cubes = iris.load('../microsoft/*.nc')
full_cube = iris.cube.CubeList(cubes).concatenate_cube()
full_cube

<iris 'Cube' of air_temperature / (degC) (time: 366; projection_y_coordinate: 290; projection_x_coordinate: 180)>

In [3]:
import cartopy

def transform_coords(cube, lon, lat):
    expected_proj = cube.coords('projection_y_coordinate')[0].coord_system.as_cartopy_crs()
    given_proj = cartopy.crs.PlateCarree()
    return expected_proj.transform_point(lon, lat, given_proj)

In [4]:
def get_avg(full_cube, lat, lon, year, month, day, past_years=56):
    x, y = transform_coords(full_cube, lon, lat)
    
    first_year = next(full_cube.slices_over('time'))

    samples = [('projection_y_coordinate', y), ('projection_x_coordinate', x)]
    first_point = first_year.interpolate(samples, iris.analysis.Nearest())
    
    exact_x = first_point.coord('projection_x_coordinate')[0].points[0]
    exact_y = first_point.coord('projection_y_coordinate')[0].points[0]
    
    x_const = iris.Constraint(projection_x_coordinate=exact_x)
    y_const = iris.Constraint(projection_y_coordinate=exact_y)

    all_points = full_cube.extract(x_const).extract(y_const)
    
    years = [year - i for i in range(past_years)]
    time_units = all_points.coord('time').units
    vals = [time_units.date2num(datetime.datetime(y, month, day)) for y in years]
    time_constraint = iris.Constraint(time=vals)
    
    day_cube = all_points.extract(time_constraint)
    
    time_mean = day_cube.collapsed('time', iris.analysis.MEAN)
    return time_mean.data

In [5]:
get_avg(full_cube, lat, lon, 1960, 1, 15, past_years=1)

array(1.4383695125579834)